### Stanford Ribonanza RNA Folding Challange
PS: I am not an an expert, I am doing some chatgpt google research to possible solutions or baseline to such problem, my background in Physics but always loved biology too. 
My main motivation to learn and provide simple starter code for people

- Tasks: Reduce memory, Use BPP files, Add folding

In [ ]:
import tqdm
import pandas as pd 
import os 
from tqdm import tqdm as tq
from collections import defaultdict
import numpy as np
import multiprocessing

let's read sample data

In [ ]:
test = pd.read_csv('/kaggle/input/stanford-ribonanza-rna-folding/test_sequences.csv')[:50000]
train = pd.read_csv('/kaggle/input/stanford-ribonanza-rna-folding/train_data.csv')[:250000]
train.head()

In [ ]:
train.info()

In [ ]:
def reduce_dataframe_size(df):
    # Make a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Define a mapping of column names to more memory-efficient data types
    column_type_mapping = {}

    for column in df_copy.columns:
        dtype = df_copy[column].dtype

        if dtype == 'object':
            # Handle object columns (e.g., strings)
            unique_values = df_copy[column].nunique()

            if unique_values == 1:
                # If a column has only one unique value, convert it to the appropriate data type
                if 'int' in dtype.name:
                    column_type_mapping[column] = 'int64'
                elif 'float' in dtype.name:
                    column_type_mapping[column] = 'float64'
                elif 'datetime' in dtype.name:
                    column_type_mapping[column] = 'datetime64'
                else:
                    column_type_mapping[column] = 'category'
            elif unique_values < len(df_copy) / 2:
                # If a column has less than half unique values, convert it to category
                column_type_mapping[column] = 'category'
        elif dtype == 'float64':
            # Reduce float64 columns to float32 or float16
            column_type_mapping[column] = 'float32'
        elif dtype == 'int64':
            # Reduce int64 columns to int32 or int16
            column_type_mapping[column] = 'int32'
        elif dtype == 'datetime64':
            # Reduce datetime64 columns to datetime32
            column_type_mapping[column] = 'datetime32'
    
    # Apply the data type conversions based on the mapping
    df_copy = df_copy.astype(column_type_mapping)

    return df_copy
reduced_train = reduce_dataframe_size(train)

In [ ]:
test.info()

In [ ]:
reduced_test = reduce_dataframe_size(test)
reduced_test.info()

In [ ]:
train.info()

In [ ]:
reduced_train.info()

In [ ]:
reduced_train

Getting txt file as a single dataframe which is base-pairing probability matrix of each sequences in our train and test dataset. Definition:
- **A base-pairing probability matrix** (BPPM) stores the probabilities for every possible base pair in an RNA sequence and has been used in many algorithms in RNA informatics (e.g., RNA secondary structure prediction and motif search). [source](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3514005/)


In [ ]:
root_dir = '/kaggle/input/stanford-ribonanza-rna-folding/Ribonanza_bpp_files/extra_data'
file_paths = []

for folder, _, files in tq(os.walk(root_dir), total=len(os.listdir(root_dir))):
    for file in files:
        file_paths.append(os.path.join(folder, file))

# Now, file_paths contains the list of all file paths within the specified directory and its subdirectories.


- filter sequences only in dataset

In [ ]:
# Get unique sequence_id values from both dataframes
unique_sequence_ids = set(reduced_train['sequence_id']) | set(reduced_test['sequence_id'])

# Initialize tqdm with the list of file paths
with tq(total=len(file_paths), desc="Filtering Files") as pbar:
    # Filter file paths using list comprehension
    filtered_file_paths = [
        filepath
        for filepath in file_paths
        if os.path.splitext(os.path.basename(filepath))[0] in unique_sequence_ids
    ]
    pbar.update(len(file_paths))  # Update the progress bar to completion

- We can get avarage of probability for each sequence_id

In [ ]:
# Create a defaultdict to store sequence_id and their corresponding probabilities
sequence_probabilities = defaultdict(list)

# Initialize tqdm with the list of text files
with tq(total=len(filtered_file_paths), desc="Processing Files") as pbar:
    # Iterate through each file path
    for filepath in filtered_file_paths:
        with open(filepath, "r") as file:
            for line in file:
                # Split each line into values
                values = line.strip().split()  # Split based on spaces

                # Ensure that there are at least 3 values (pair1, pair2, probability)
                if len(values) < 3:
                    continue

                # Extract pair1, pair2, and probability
                _, _, probability = values[:3]

                # Use the filename (without extension) as the key
                sequence_id = os.path.splitext(os.path.basename(filepath))[0]

                # Append the probability to the list for the sequence_id
                sequence_probabilities[sequence_id].append(float(probability))

        pbar.update(1)  # Update the progress bar

# Calculate the average probability for each sequence_id
average_probabilities = {}
for sequence_id, probabilities in sequence_probabilities.items():
    average_prob = sum(probabilities) / len(probabilities)
    average_probabilities[sequence_id] = average_prob

delete some memory

In [ ]:
del train
del test
del file_paths
del filtered_file_paths

- Adding average probability to train and test dataset and drop null columns

In [ ]:
avg_p = pd.DataFrame(average_probabilities, index=[0]).T.reset_index()
avg_p.columns = ['sequence_id', 'avg_probability']
reduced_train_new  = reduced_train.merge(avg_p, on='sequence_id', how='left').dropna(axis=1,how='all')
reduced_test_new  = reduced_test.merge(avg_p, on='sequence_id', how='left').dropna(axis=1,how='all')

In [ ]:
del reduced_train
del reduced_test
del average_probabilities
del avg_p

In [ ]:
reduced_train_new

In [ ]:
reduced_test_new

- Folding RNA structures, using swifter for faster apply.

In [ ]:
!pip install viennarna
!pip install swifter

In [ ]:
from ViennaRNA import fold
import swifter


# Apply complex_structure_features calculation in parallel with a progress bar
def process_with_progress(df):
    with tq(total=len(df), desc="Processing Data") as pbar:
        df['complex_structure_features'] = df['sequence'].swifter.apply(fold)
        pbar.update(len(df))  # Update the progress bar to completion

# Apply the processing function to your dataframes
process_with_progress(reduced_train_new)
process_with_progress(reduced_test_new)

In [ ]:
# Function to count parentheses
def count_parentheses(structure_string):
    count = structure_string.count(")")
    return count

# Apply the function to the DataFrame column
tq.pandas()
reduced_train_new['parentheses_counts'] = reduced_train_new['complex_structure_features'].astype(str).apply(count_parentheses)


In [ ]:
reduced_train_new

In [ ]:
!pip install forgi

In [ ]:
import forgi.visual.mplotlib as fvm
import forgi.graph.bulge_graph as fgb
import random
import matplotlib.pyplot as plt

# Assuming you have a DataFrame called reduced_train_new with the structures and sequences

# Number of random structures to plot
num_structures_to_plot = 5

# Get random indices from your DataFrame
random_indices = random.sample(range(len(reduced_train_new)), num_structures_to_plot)

# Create subplots for each structure
plt.figure(figsize=(15, 5 * num_structures_to_plot))

for i, idx in enumerate(random_indices, 1):
    structure = reduced_train_new['complex_structure_features'][idx][0]
    sequence = reduced_train_new['sequence'][idx]

    plt.subplot(num_structures_to_plot, 1, i)
    bg = fgb.BulgeGraph.from_fasta_text(f'>rna{idx}\n{structure}\n{sequence}')[0]
    fvm.plot_rna(bg, lighten=0.5, text_kwargs={"fontweight": None})
    plt.title(f"RNA Structure {idx}")

plt.tight_layout()
plt.show()